### This scipt helps to take inventory of the source Tableau Server. Code snippets below will print list of workbooks, datasources and users with their respective projects and groups. 

In [ ]:
import tableauserverclient as TSC
import getpass
import pandas as pd

In [ ]:
# Connect to Source Server

source_ts = '<<Source Server>>'
source_user = '<<Tableau Admin User>>'
source_site = '<<Tableau Online Site>>' # For Tablesu Online Only

source_pwd = getpass.getpass("Enter password of source Tableau Server: ")
source_tableau_auth = TSC.TableauAuth(source_user, source_pwd, source_site)
source_server = TSC.Server(source_ts,use_server_version=True)
with source_server.auth.sign_in(source_tableau_auth): 
    print("Connected to Source Server")

##### Produce a CSV of SourceWorkbookName|WorkbookProject|ParentProject

In [ ]:
# Create Project_ID -> Project_Name pair
source_projects = {}
with source_server.auth.sign_in(source_tableau_auth): 
        # get all projects on site
        all_project_items, pagination_item = source_server.projects.get()
        for proj in all_project_items:
            source_projects[proj.id] = proj.name

In [ ]:
# Create Project_ID -> Project_Name + Parent_Project (If Any)
#print('SubProjectName | ParentProjectName')

with source_server.auth.sign_in(source_tableau_auth): 
        # get all projects on site
        all_project_items, pagination_item = source_server.projects.get()
        for proj in all_project_items:
            if proj.parent_id:
                #print(proj.name, "|" , source_projects[proj.parent_id])
                source_projects[proj.id] = proj.name + '|' + source_projects[proj.parent_id]
            else:
                source_projects[proj.id] = proj.name
#print(source_projects)

In [ ]:
###### change below to produce a CSV file instead

print('SourceWorkbookName|WorkbookProject|ParentProject')
with source_server.auth.sign_in(source_tableau_auth): 
    all_workbooks, pagination_item = source_server.workbooks.get()
    for workbook in all_workbooks:
            print(workbook.name + '|' + source_projects[workbook.project_id])

##### Produce a CSV of SourceDataSourceName|SourceSubProject|SourceProject

In [ ]:
###### change below to produce a CSV file instead

print('SourceDatasourceName|DatasourceProject|ParentProject')
with source_server.auth.sign_in(source_tableau_auth):
    all_datasources, pagination_item = source_server.datasources.get()
    for datasource in all_datasources:
        print(datasource.name + '|' + source_projects[datasource.project_id])

##### Produce a CSV of SourceUsername|SourceSiteRole

In [ ]:
###### change below to produce a CSV file instead

print("SourceUsername|SourceSiteRole")
with source_server.auth.sign_in(source_tableau_auth):
    all_users, pagination_item = source_server.users.get()
    for user in all_users:
        print(user.name + "|" + user.site_role)
